In [73]:
import requests, os, pickle
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime


url = "https://nookipedia.com/wiki/Villager/New_Horizons"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to load page with status code: {response.status_code}")

soup = BeautifulSoup(response.content, 'html.parser')

In [14]:
table = soup.find('table')
rows = table.find_all('tr', {"style" : "text-align:center;"})

characters = []

os.makedirs("villager_images", exist_ok=True)

for r in rows:
  
  img_tag = r.find('img')
  if img_tag:
    # Extract image URL (src) and image description (alt)
    img_url = img_tag['src'] if 'src' in img_tag.attrs else None
    img_alt = img_tag['alt'] if 'alt' in img_tag.attrs else None

    if img_url:
      img_response = requests.get(img_url)
      if img_response.status_code == 200:
        img_filename = f'villager_images/{img_alt}'
        with open(img_filename, 'wb') as img_file:
          img_file.write(img_response.content)

        characters.append({'Name': img_alt, 'Image path': img_filename})

In [65]:
images_df = pd.DataFrame(characters)

In [67]:
images_df['Name'] = images_df['Name'].str.split("'s").str[0]

In [85]:
re = images_df.loc[images_df['Name'].str.contains('Wart')]

HTML(re.to_html(escape=False))

,Name,Image path,Image
402,Wart Jr.,villager_images/Wart Jr.'s Photo NH Texture.png,


In [69]:
from IPython.display import HTML

def path_to_image_html(path):
    return f'<img src="{path}" width="50" height="50">'

images_df['Image'] = images_df['Image path'].apply(path_to_image_html)

HTML(images_df.to_html(escape=False))



,Name,Image path,Image
0,Ace,villager_images/Ace's Photo NH Texture.png,
1,Admiral,villager_images/Admiral's Photo NH Texture.png,
2,Agent S,villager_images/Agent S's Photo NH Texture.png,
3,Agnes,villager_images/Agnes's Photo NH Texture.png,
4,Al,villager_images/Al's Photo NH Texture.png,
5,Alfonso,villager_images/Alfonso's Photo NH Texture.png,
6,Alice,villager_images/Alice's Photo NH Texture.png,
7,Alli,villager_images/Alli's Photo NH Texture.png,
8,Amelia,villager_images/Amelia's Photo NH Texture.png,
9,Anabelle,villager_images/Anabelle's Photo NH Texture.png,


In [74]:
images_df.to_pickle("images_df.pkl")

In [71]:
# Save the DataFrame with image tags as an HTML file
images_df.to_html('villagers_with_images.html', escape=False)